In [ ]:
!pip install transformers datasets tqdm torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


Training BART Model On PAWS Dataset

In [ ]:
from datasets import load_dataset
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
import torch

# Load PAWS dataset from Hugging Face
dataset = load_dataset("paws", "labeled_final")

# Inspect dataset
print(dataset)

# Preprocess Dataset
def preprocess_function(examples):
    tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")
    inputs = examples["sentence1"]  # Sentence 1
    targets = examples["sentence2"]  # Sentence 2
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length").input_ids
    model_inputs["labels"] = [label if label is not None else -100 for label in labels]  # Ensure proper label formatting
    return model_inputs

# Tokenize Dataset
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Load Model
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")

# Define Training Arguments
training_args = TrainingArguments(
    output_dir="./bart_paws_model",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=500,
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",  # Disable W&B logging
    metric_for_best_model="loss",
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=lambda data: {
        "input_ids": torch.tensor([item["input_ids"] for item in data]),
        "attention_mask": torch.tensor([item["attention_mask"] for item in data]),
        "labels": torch.tensor([item["labels"] for item in data])
    },
)

# Train Model
trainer.train()

# Save Model
model.save_pretrained("./bart_paws_model")
tokenizer.save_pretrained("./bart_paws_model")

# Function for Paraphrasing
def paraphrase(input_text, num_return_sequences=1, num_beams=4):
    tokenizer = BartTokenizer.from_pretrained("./bart_paws_model")
    model = BartForConditionalGeneration.from_pretrained("./bart_paws_model")
    inputs = tokenizer([input_text], max_length=128, truncation=True, return_tensors="pt")
    outputs = model.generate(
        inputs["input_ids"],
        max_length=128,
        num_beams=num_beams,
        num_return_sequences=num_return_sequences,
        early_stopping=True
    )
    return [tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True) for output in outputs]

# Example Usage
input_text = "The quick brown fox jumps over the lazy dog."
paraphrased_texts = paraphrase(input_text, num_return_sequences=3)
print("Original:", input_text)
print("Paraphrased:", paraphrased_texts)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.79k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/8.43M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.24M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

...

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss
1,0.111200,0.145548
2,0.075500,0.143283
3,0.053800,0.141280


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


Original: The quick brown fox jumps over the lazy dog.
Paraphrased: ['The lazy fox jumps over the quick brown dog.', 'The lazy fox jumps over the quick brown dog.', 'The lazy fox jumps over the fast brown dog.']


In [ ]:
model.save_pretrained("./bart_paws_final_model")
tokenizer.save_pretrained("./bart_paws_final_model")

('./bart_paws_final_model/tokenizer_config.json',
 './bart_paws_final_model/special_tokens_map.json',
 './bart_paws_final_model/vocab.json',
 './bart_paws_final_model/merges.txt',
 './bart_paws_final_model/added_tokens.json')

In [ ]:
import shutil

# Zip the directory
shutil.make_archive("bart_paws_final_model", "zip", "./bart_paws_final_model")


'/content/bart_paws_final_model.zip'

-------------------------------------------------------------------------------------END------------------------------------------------------------------------
